# Los Angeles Crime Dataset (Exploration,Cleaning,Transformations)

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
data = pd.read_csv('Crime_Data_from_2010_to_2019_20250408.csv')

In [3]:
### 📊 *Explanation of Each Column:*

# | *Column Name*          | *What It Means*                                          |
# |--------------------------|------------------------------------------------------------|
# | *DR_NO*                | *Report Number* – A unique identifier for the crime report. |
# | *Date Rptd*            | *Date Reported* – The date when the crime report was made. |
# | *DATE OCC*             | *Date of Occurrence* – The date when the crime actually happened. |
# | *TIME OCC*             | *Time of Occurrence* – The time when the crime took place (in 24-hour format). |
# | *AREA*                 | *Area Number* – A code that represents the area within Los Angeles where the crime happened. |
# | *AREA NAME*            | *Area Name* – The name of the area or neighborhood where the crime occurred (e.g., Hollywood, Downtown). |
# | *Rpt Dist No*          | *Reporting District Number* – A more specific location within the area (a police district code). |
# | *Part 1-2*             | *Part 1 and Part 2 Crimes* – Differentiates between serious crimes (Part 1) like homicide and less serious crimes (Part 2) like vandalism. |
# | *Crm Cd*               | *Crime Code* – A numeric code that corresponds to a specific crime type (e.g., theft, assault). |
# | *Crm Cd Desc*          | *Crime Description* – A short description of the crime (e.g., burglary, robbery). |
# | *Mocodes*              | *Modus Operandi Codes* – Codes that describe the method or pattern of the crime (e.g., how the crime was committed). |
# | *Vict Age*             | *Victim Age* – The age of the person who was the victim of the crime. |
# | *Vict Sex*             | *Victim Sex* – The gender of the victim (e.g., Male, Female). |
# | *Vict Descent*         | *Victim Descent* – The ethnicity or racial background of the victim (e.g., White, Black, Hispanic). |
# | *Premis Cd*            | *Premises Code* – A code that describes the type of location where the crime occurred (e.g., residence, street). |
# | *Premis Desc*          | *Premises Description* – A description of the location where the crime took place (e.g., home, office, parking lot). |
# | *Weapon Used Cd*       | *Weapon Used Code* – A code representing the type of weapon used in the crime (e.g., gun, knife, no weapon). |
# | *Weapon Desc*          | *Weapon Description* – A description of the weapon used in the crime (e.g., handgun, bat, or "no weapon"). |
# | *Status*               | *Crime Status* – The current status of the case (e.g., open, closed, under investigation). |
# | *Status Desc*          | *Status Description* – A more detailed description of the case status (e.g., "Arrest made," "Investigation ongoing"). |
# | *Crm Cd 1, 2, 3, 4*     | *Additional Crime Codes* – If the incident involved multiple crimes, these codes refer to other crimes related to the primary crime. |
# | *LOCATION*             | *Crime Location* – The exact street address or neighborhood where the crime occurred (sometimes general). |
# | *Cross Street*         | *Cross Street* – The nearest intersection or cross street to where the crime took place. |
# | *LAT*                  | *Latitude* – The latitude coordinate of the crime location (for mapping purposes). |
# | *LON*                  | *Longitude* – The longitude coordinate of the crime location (for mapping purposes). |

# ---

### Filter Data From 2015-2019

#### Convert the 'DATE OCC' col to datetime

In [4]:
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')

In [5]:
data['DATE OCC'].head()

0   2010-02-20
1   2010-09-12
2   2010-08-09
3   2010-01-05
4   2010-01-02
Name: DATE OCC, dtype: datetime64[ns]

In [6]:
la_data = data[(data['DATE OCC'].dt.year >= 2015) & (data['DATE OCC'].dt.year <= 2019)]

### Drop Unnecessary Columns From the Data 

In [7]:
# convert col_names into lower and strip them 
la_data.columns = la_data.columns.str.lower().str.strip()
la_data.columns

Index(['dr_no', 'date rptd', 'date occ', 'time occ', 'area', 'area name',
       'rpt dist no', 'part 1-2', 'crm cd', 'crm cd desc', 'mocodes',
       'vict age', 'vict sex', 'vict descent', 'premis cd', 'premis desc',
       'weapon used cd', 'weapon desc', 'status', 'status desc', 'crm cd 1',
       'crm cd 2', 'crm cd 3', 'crm cd 4', 'location', 'cross street', 'lat',
       'lon'],
      dtype='object')

In [8]:
la_data = la_data.drop(columns=['crm cd 1',
 'crm cd 2',
 'crm cd 3',
 'crm cd 4',
 'crm cd',
 'cross street',
 'lat',
 'lon',
 'weapon used cd',
 'mocodes',
 'premis cd',
 'rpt dist no',
 'area'])

In [9]:
la_data.shape

(1197671, 15)

### Duplicate Records 

In [10]:
la_data.duplicated(subset = "dr_no",keep="first").value_counts()

False    1081884
True      115787
Name: count, dtype: int64

In [11]:
# get the duplicate records 
la_data[la_data.duplicated(subset="dr_no",keep=False)].sort_values(by="dr_no")

,dr_no,date rptd,date occ,time occ,area name,part 1-2,crm cd desc,vict age,vict sex,vict descent,premis desc,weapon desc,status,status desc,location
1130670,161206510,02/05/2016 12:00:00 AM,2016-02-05,700,77th Street,1,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",51,M,H,AUTO SALES LOT,NaN,IC,Invest Cont,900 W FLORENCE AV
1356399,161206510,02/05/2016 12:00:00 AM,2016-02-05,700,77th Street,1,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",51,M,H,AUTO SALES LOT,NaN,IC,Invest Cont,900 W FLORENCE AV
1120980,161206511,02/06/2016 12:00:00 AM,2016-02-06,620,77th Street,2,VANDALISM - MISDEAMEANOR ($399 OR UNDER),29,M,H,RESTAURANT/FAST FOOD,NaN,IC,Invest Cont,100 W MANCHESTER AV
1346672,161206511,02/06/2016 12:00:00 AM,2016-02-06,620,77th Street,2,VANDALISM - MISDEAMEANOR ($399 OR UNDER),29,M,H,RESTAURANT/FAST FOOD,NaN,IC,Invest Cont,100 W MANCHESTER AV
1191452,161206512,02/05/2016 12:00:00 AM,2016-02-05,700,77th Street,1,BURGLARY,30,M,H,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,IC,Invest Cont,800 W 54TH ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046971,242106553,03/04/2024 12:00:00 AM,2016-03-25,1200,Topanga,2,OTHER MISCELLANEOUS CRIME,19,F,A,SINGLE FAMILY DWELLING,NaN,IC,Invest Cont,22500 SCHOOLCRAFT ST
1237851,242106857,03/13/2024 12:00:00 AM,2016-01-13,1526,Topanga,2,THEFT OF IDENTITY,25,F,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,IC,Invest Cont,6700 INDEPENDENCE AV
1463848,242106857,03/13/2024 12:00:00 AM,2016-01-13,1526,Topanga,2,THEFT OF IDENTITY,25,F,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,IC,Invest Cont,6700 INDEPENDENCE AV
1199684,242108843,05/06/2024 12:00:00 AM,2016-05-27,655,Topanga,2,THEFT OF IDENTITY,39,F,O,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,IC,Invest Cont,6200 DE SOTO AV


In [12]:
la_data = la_data.drop_duplicates(subset="dr_no",keep="first")

### Exploring and Cleaning 

In [13]:
la_data.shape

(1081884, 15)

In [14]:
la_data.isnull().sum()

dr_no                0
date rptd            0
date occ             0
time occ             0
area name            0
part 1-2             0
crm cd desc          0
vict age             0
vict sex        101880
vict descent    101906
premis desc        149
weapon desc     712957
status               1
status desc          0
location             0
dtype: int64

##### Vict Age column

In [15]:
la_data["vict age"].value_counts().sort_values()

vict age
 118         1
-8           2
-7           2
-6           3
-5          11
         ...  
 28      24008
 27      24184
 25      24206
 26      24374
 0      199123
Name: count, Length: 108, dtype: int64

In [16]:
# Age col containing value equal to 0 or less which donot make sense 
la_data[la_data["vict age"] <= 0]["vict age"].value_counts()

vict age
 0    199123
-1       141
-2        71
-3        55
-4        17
-5        11
-6         3
-8         2
-7         2
Name: count, dtype: int64

In [17]:
# Replacing all values which are 0 or less than with UnKnown 

In [18]:
la_data["vict age"] = la_data["vict age"].apply(lambda x: x if x > 0 else "UnKnown")

In [19]:
la_data["vict age"].value_counts()

vict age
UnKnown    199425
26          24374
25          24206
27          24184
28          24008
            ...  
95            145
96            108
97            102
98             80
118             1
Name: count, Length: 100, dtype: int64

In [20]:
la_data["vict age"].isnull().sum()

np.int64(0)

##### Vict Sex Column

In [21]:
la_data["vict sex"].isnull().sum()

np.int64(101880)

In [22]:
la_data["vict sex"].value_counts()

vict sex
M    489285
F    441698
X     48969
H        35
N        17
Name: count, dtype: int64

In [23]:
#  In the dataset documentation there is only (F - Female M - Male X - Unknown) so i replace H & N with X

In [24]:
la_data["vict sex"] = la_data["vict sex"].replace({"H":"X","N":"X"})

In [25]:
la_data["vict sex"].value_counts()

vict sex
M    489285
F    441698
X     49021
Name: count, dtype: int64

In [26]:
# Replacing all the Missing value with X
la_data["vict sex"] = la_data["vict sex"].fillna("X")

In [27]:
la_data["vict sex"].isnull().sum()

np.int64(0)

##### Vict Descent column

In [28]:
la_data["vict descent"].isnull().sum()

np.int64(101906)

In [29]:
la_data["vict descent"].unique()

array(['H', 'B', 'O', 'W', nan, 'X', 'A', 'K', 'V', 'F', 'C', 'G', 'I',
       'J', 'P', 'U', 'S', 'Z', 'L', 'D', '-'], dtype=object)

In [30]:
descent_mapping = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian',
    '-': 'Unknown'
}
la_data["vict descent"] = la_data["vict descent"].map(descent_mapping)

In [31]:
# Replacing all the missing values with UnKnown
la_data["vict descent"] = la_data["vict descent"].fillna("UnKnown")

In [32]:
la_data["vict descent"].isnull().sum()

np.int64(0)

In [33]:
la_data["vict descent"].value_counts()

vict descent
Hispanic/Latin/Mexican            365639
White                             248525
Black                             163877
Other                             105344
UnKnown                           101906
Unknown                            60379
Other Asian                        28199
Korean                              4566
Filipino                            1256
Chinese                              749
American Indian/Alaskan Native       559
Japanese                             254
Pacific Islander                     183
Vietnamese                           151
Hawaiian                             103
Asian Indian                         102
Guamanian                             53
Samoan                                16
Laotian                               12
Cambodian                             11
Name: count, dtype: int64

In [34]:
# above i got two separate UnKnown and both are same so, we replace it
la_data["vict descent"] = la_data["vict descent"].replace("Unknown","UnKnown")

In [35]:
la_data["vict descent"].value_counts()

vict descent
Hispanic/Latin/Mexican            365639
White                             248525
Black                             163877
UnKnown                           162285
Other                             105344
Other Asian                        28199
Korean                              4566
Filipino                            1256
Chinese                              749
American Indian/Alaskan Native       559
Japanese                             254
Pacific Islander                     183
Vietnamese                           151
Hawaiian                             103
Asian Indian                         102
Guamanian                             53
Samoan                                16
Laotian                               12
Cambodian                             11
Name: count, dtype: int64

##### Premis Desc Column

In [36]:
la_data["premis desc"].isnull().sum()

np.int64(149)

In [37]:
la_data["premis desc"].value_counts().head(60)

premis desc
STREET                                                      244916
SINGLE FAMILY DWELLING                                      205305
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)                127823
PARKING LOT                                                  79034
SIDEWALK                                                     54335
OTHER BUSINESS                                               48102
VEHICLE, PASSENGER/TRUCK                                     40540
DRIVEWAY                                                     22347
GARAGE/CARPORT                                               19030
RESTAURANT/FAST FOOD                                         16752
DEPARTMENT STORE                                             14273
MARKET                                                       11202
PARKING UNDERGROUND/BUILDING                                 10095
OTHER STORE                                                   8735
PARK/PLAYGROUND                                   

In [38]:
la_data["premis desc"] = la_data["premis desc"].fillna('UnKnown')

In [39]:
la_data["premis desc"].isnull().sum()

np.int64(0)

In [40]:
la_data["premis desc"] = la_data["premis desc"].str.strip().str.upper()

In [41]:
la_data["premis desc"].value_counts()

premis desc
STREET                                          244916
SINGLE FAMILY DWELLING                          205305
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)    127823
PARKING LOT                                      79034
SIDEWALK                                         54335
                                                 ...  
MTA - ORANGE LINE - WARNER CTR                       2
MTA - ORANGE LINE - WOODMAN                          2
MTA - SILVER LINE - LAC/USC MEDICAL CENTER           2
REDLINE SUBWAY RAIL CAR (INSIDE TRAIN)               1
NUCLEAR FACILITY                                     1
Name: count, Length: 315, dtype: int64

##### Weapon Desc Column

In [42]:
la_data["weapon desc"].value_counts()

weapon desc
STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)    220365
UNKNOWN WEAPON/OTHER WEAPON                        32095
VERBAL THREAT                                      28655
HAND GUN                                           17799
KNIFE WITH BLADE 6INCHES OR LESS                    7107
                                                   ...  
ANTIQUE FIREARM                                        5
UZI SEMIAUTOMATIC ASSAULT RIFLE                        4
M1-1 SEMIAUTOMATIC ASSAULT RIFLE                       3
HECKLER & KOCH 91 SEMIAUTOMATIC ASSAULT RIFLE          2
MAC-11 SEMIAUTOMATIC ASSAULT WEAPON                    2
Name: count, Length: 78, dtype: int64

In [43]:
la_data["weapon desc"].isnull().sum()

np.int64(712957)

In [44]:
# too many missing values 
res = la_data[la_data["weapon desc"].isnull()][["crm cd desc","weapon desc"]]
res["crm cd desc"].value_counts().head(60)

crm cd desc
VEHICLE - STOLEN                                            82135
BURGLARY FROM VEHICLE                                       77450
THEFT PLAIN - PETTY ($950 & UNDER)                          70682
BURGLARY                                                    64191
THEFT OF IDENTITY                                           60331
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)     56037
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)             49701
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                    36756
THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD    33478
SHOPLIFTING - PETTY THEFT ($950 & UNDER)                    28680
THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)         17367
TRESPASSING                                                 13129
BIKE - STOLEN                                               10991
VIOLATION OF RESTRAINING ORDER                              10132
DOCUMENT FORGERY / STOLEN FELONY                             994

### Why weapon desc col contain too many missing values?
##### Most of the crimes with missing weapon descriptions are non-violent crimes like:
##### Theft and burglary (e.g., stolen cars, shoplifting, identity theft)
##### Fraud and computer crimes (e.g., forgery, embezzlement, hacking)
##### Court and restraining order violations
##### Sex-related offenses that usually don’t involve weapons
##### This tells us that in many of these crimes, weapons were not used or not reported because they were not needed to commit the crime.
##### So, the missing weapon info makes sense — it mostly happens in cases where a weapon was likely not involved.

DECISION:
Since most of the crimes in the dataset (like theft, identity fraud, vandalism, etc.) typically do not involve weapons,
we observed that the weapon_desc column has a large number of missing values
To handle this in a meaningful way:
We decided to create a new column called weapon_used, which will indicate whether a weapon was used in the crime (Yes or No).
We also replaced missing values in weapon_desc with "No Weapon" for consistency
and to avoid issues in machine learning models that don't accept null values.

In [45]:
# create new col name "weapon used"
la_data["weapon used"] = la_data["weapon desc"].notna().map({True:"Yes",False:"No"})

In [46]:
la_data["weapon used"].value_counts()

weapon used
No     712957
Yes    368927
Name: count, dtype: int64

In [47]:
# Replacing null values with "No Weapon"
la_data["weapon desc"] = la_data["weapon desc"].fillna("No Weapon")

In [48]:
la_data["weapon desc"].value_counts()

weapon desc
No Weapon                                         712957
STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)    220365
UNKNOWN WEAPON/OTHER WEAPON                        32095
VERBAL THREAT                                      28655
HAND GUN                                           17799
                                                   ...  
ANTIQUE FIREARM                                        5
UZI SEMIAUTOMATIC ASSAULT RIFLE                        4
M1-1 SEMIAUTOMATIC ASSAULT RIFLE                       3
HECKLER & KOCH 91 SEMIAUTOMATIC ASSAULT RIFLE          2
MAC-11 SEMIAUTOMATIC ASSAULT WEAPON                    2
Name: count, Length: 79, dtype: int64

In [49]:
la_data["weapon desc"].isnull().sum()

np.int64(0)

To simplify analysis, we created a new column called weapon_category by grouping specific weapons from weapon_desc into broader types like "Firearm", "Sharp Object", "Blunt Object", etc. This makes it easier to analyze trends and visualize data without dealing with too many unique weapon types.

In [50]:
def weapon_category(weapon):
    
    # check isnull or not
    if pd.isnull(weapon):
        return "UnKnown"

    # change into uppercase 
    weapon = weapon.upper()

    if any(word in weapon for word in [
        "HAND GUN", "SEMI-AUTOMATIC", "REVOLVER", "RIFLE", "SHOTGUN", "BB GUN", 
        "ASSAULT WEAPON", "FIREARM", "TOY GUN", "STARTER PISTOL", "SIMULATED GUN","AUTOMATIC WEAPON","SUB-MACHINE GUN"]):
        return "Firearm"

    elif any(word in weapon for word in ["KNIFE", "BLADE", "MACHETE", "SCISSORS", "SCREWDRIVER", "CLEAVER", 
        "STRAIGHT RAZOR", "RAZOR", "ICE PICK", "DAGGER", "SWORD"]):
        return "Knife/Sharp Object"

    elif any(word in weapon for word in ["STICK", "BAT", "PIPE", "INSTRUMENT", "HAMMER", "BOARD", "TIRE IRON", 
        "AXE", "BRICK", "BLACKJACK", "BLOCK", "BELT", "CHAIN","BOTTLE","ROCK/THROWN OBJECT","FIXED OBJECT","BRASS KNUCKLES"]):
        return "Blunt Object"

    elif any(word in weapon for word in ["STRONG-ARM", "VERBAL THREAT", "PHYSICAL PRESENCE", "DOG", "ANIMAL","No Weapon"]):
        return "UnArmed/Bodily Force"

    elif any(word in weapon for word in ["BOMB", "FIRE", "CHEMICAL", "EXPLOXIVE", "SCALDING","MACE/PEPPER SPRAY"]):
        return "Explosive/Fire/Chemical"

    elif any(word in weapon for word in ["VEHICLE", "SYRINGE", "DEMAND NOTE", "LIQUOR", "ROPE", "GLASS", 
        "MARTIAL ARTS", "BOW", "UNKNOWN", "OTHER", "STUN GUN"]):
        return "Other"

    else:
        return "UnKnown"

In [51]:
la_data["weapon_category"] = la_data["weapon desc"].apply(weapon_category)

In [52]:
la_data.head()

,dr_no,date rptd,date occ,time occ,area name,part 1-2,crm cd desc,vict age,vict sex,vict descent,premis desc,weapon desc,status,status desc,location,weapon used,weapon_category
804945,152005022,01/21/2015 12:00:00 AM,2015-01-21,1415,Olympic,1,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",28,F,Hispanic/Latin/Mexican,STREET,HAND GUN,IC,Invest Cont,1000 S MARIPOSA AV,Yes,Firearm
804947,160605536,01/27/2016 12:00:00 AM,2015-06-01,800,Hollywood,2,THEFT OF IDENTITY,50,F,Hispanic/Latin/Mexican,SINGLE FAMILY DWELLING,No Weapon,IC,Invest Cont,1400 N DETROIT ST,No,UnKnown
804948,160711213,06/15/2016 12:00:00 AM,2015-12-15,945,Wilshire,2,THEFT OF IDENTITY,51,M,Black,SINGLE FAMILY DWELLING,No Weapon,IC,Invest Cont,5300 W OLYMPIC BL,No,UnKnown
804952,152118490,11/10/2015 12:00:00 AM,2015-11-09,1930,Topanga,1,BURGLARY FROM VEHICLE,32,M,Other,STREET,No Weapon,IC,Invest Cont,21300 PARTHENIA ST,No,UnKnown
804953,151717560,10/15/2015 12:00:00 AM,2015-10-15,1445,Devonshire,2,THROWING OBJECT AT MOVING VEHICLE,48,F,White,STREET,No Weapon,IC,Invest Cont,BALBOA,No,UnKnown


In [53]:
la_data["weapon_category"].value_counts()

weapon_category
UnKnown                    712957
UnArmed/Bodily Force       250229
Other                       36978
Firearm                     35558
Knife/Sharp Object          22089
Blunt Object                20357
Explosive/Fire/Chemical      3716
Name: count, dtype: int64

In [54]:
# Check we did correct categorization
# here "UnKnown" value_count is same as "No Weapon" value_count which indicate correct categorization
la_data[la_data["weapon_category"] == "UnKnown"]["weapon desc"].value_counts()

weapon desc
No Weapon    712957
Name: count, dtype: int64

In [55]:
la_data[la_data["weapon desc"] == "No Weapon"].shape
# 712957

(712957, 17)

##### Status Column

In [56]:
# IC → Investigation Continued
# AO → Adult Other (case referred or handled outside LAPD court process)
# AA → Adult Arrest
# JA → Juvenile Arrest
# JO → Juvenile Other (case referred or handled differently for juveniles)
# CC → Cleared-Commission/Non-LAPD (closed by outside agency)
# UNK → Unknown

# AA, JA, CC → resolved
# IC, AO, JO, UNK → unresolved

In [57]:
la_data[la_data["status"].isnull()]

,dr_no,date rptd,date occ,time occ,area name,part 1-2,crm cd desc,vict age,vict sex,vict descent,premis desc,weapon desc,status,status desc,location,weapon used,weapon_category
1983937,191307542,03/06/2019 12:00:00 AM,2019-03-06,1645,Newton,1,VEHICLE - STOLEN,UnKnown,X,UnKnown,UNKNOWN,No Weapon,NaN,UNK,STANFORD,No,UnKnown


In [58]:
la_data["status"] = la_data["status"].fillna("UNK")

In [59]:
la_data["status"].isnull().sum()

np.int64(0)

In [60]:
la_data["status"].value_counts()

status
IC     819350
AO     138463
AA     113605
JA       7585
JO       2870
CC          9
19          1
UNK         1
Name: count, dtype: int64

In [61]:
la_data[la_data["status"] == '19']

,dr_no,date rptd,date occ,time occ,area name,part 1-2,crm cd desc,vict age,vict sex,vict descent,premis desc,weapon desc,status,status desc,location,weapon used,weapon_category
1167320,161922607,11/30/2016 12:00:00 AM,2016-11-25,950,Mission,1,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),UnKnown,X,UnKnown,STREET,No Weapon,19,UNK,15800 OLDEN ST,No,UnKnown


In [62]:
# replace 19 with unknown
la_data["status"] = la_data["status"].replace("19","UNK")

In [63]:
la_data["status"].value_counts()

status
IC     819350
AO     138463
AA     113605
JA       7585
JO       2870
CC          9
UNK         2
Name: count, dtype: int64

In [64]:
la_data[la_data["status"] == "CC"]
# the case was closed without further police involvement or clear resolution.(CC means)

,dr_no,date rptd,date occ,time occ,area name,part 1-2,crm cd desc,vict age,vict sex,vict descent,premis desc,weapon desc,status,status desc,location,weapon used,weapon_category
1287487,160217814,09/27/2016 12:00:00 AM,2016-09-26,1645,Rampart,2,"BUNCO, PETTY THEFT",18,M,Hispanic/Latin/Mexican,SINGLE FAMILY DWELLING,No Weapon,CC,UNK,1800 MARYLAND ST,No,UnKnown
1311831,160327645,12/20/2016 12:00:00 AM,2016-12-20,1350,Southwest,1,SHOPLIFTING - PETTY THEFT ($950 & UNDER),23,M,Hispanic/Latin/Mexican,MARKET,No Weapon,CC,UNK,5000 RODEO RD,No,UnKnown
1544598,170314115,06/01/2017 12:00:00 AM,2017-06-01,30,Southwest,2,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",22,F,Hispanic/Latin/Mexican,STREET,No Weapon,CC,UNK,3100 RAYMOND AV,No,UnKnown
1660178,171221278,08/25/2017 12:00:00 AM,2017-07-29,1400,77th Street,1,THEFT PLAIN - PETTY ($950 & UNDER),18,F,Black,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",No Weapon,CC,UNK,6500 S NORMANDIE AV,No,UnKnown
1907645,181105575,02/06/2018 12:00:00 AM,2018-02-05,1720,Northeast,2,VANDALISM - MISDEAMEANOR ($399 OR UNDER),UnKnown,X,UnKnown,OTHER BUSINESS,No Weapon,CC,UNK,1400 ECHO PARK AV,No,UnKnown
1995988,192117351,10/12/2019 12:00:00 AM,2019-10-12,1700,Topanga,1,THEFT PLAIN - PETTY ($950 & UNDER),36,M,Other,OTHER BUSINESS,No Weapon,CC,UNK,6900 TOPANGA CANYON BL,No,UnKnown
2005323,201604049,01/02/2020 12:00:00 AM,2019-11-20,1530,Foothill,2,CHILD STEALING,UnKnown,X,UnKnown,UNKNOWN,No Weapon,CC,UNK,13700 DESMOND ST,No,UnKnown
2010643,191421123,09/29/2019 12:00:00 AM,2019-09-28,2330,Pacific,1,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",38,M,Other,PARKING UNDERGROUND/BUILDING,No Weapon,CC,UNK,5800 SEAWALK DR,No,UnKnown
2012625,201604048,01/02/2020 12:00:00 AM,2019-11-20,1530,Foothill,2,CHILD STEALING,UnKnown,X,UnKnown,UNKNOWN,No Weapon,CC,UNK,13700 DESMOND ST,No,UnKnown


##### Part 1-2 Column

In [65]:
# convert 1 with "serious" and 2 with 'normal"

In [66]:
la_data["part 1-2"].value_counts()

part 1-2
1    613605
2    468279
Name: count, dtype: int64

In [67]:
la_data["part 1-2"].isnull().sum()

np.int64(0)

In [68]:
case_mapping = {1:"Serious",2:"Normal"}
la_data["part 1-2"] = la_data["part 1-2"].map(case_mapping)

In [69]:
la_data["part 1-2"].value_counts()

part 1-2
Serious    613605
Normal     468279
Name: count, dtype: int64

##### Location Column

In [70]:
la_data["location"].isnull().sum()

np.int64(0)

In [71]:
la_data["location"].value_counts().head(60)

location
6TH                          ST             2571
7TH                          ST             2165
9300    TAMPA                        AV     2069
6TH                                         1954
7TH                                         1880
5TH                          ST             1843
5TH                                         1768
6600    TOPANGA CANYON               BL     1717
VERMONT                      AV             1576
WESTERN                      AV             1445
6800    HOLLYWOOD                    BL     1414
500 S  SAN PEDRO                    ST      1398
14000    RIVERSIDE                    DR    1397
BROADWAY                                    1393
HOLLYWOOD                                   1390
3RD                          ST             1285
700 W  7TH                          ST      1278
FIGUEROA                                    1183
VERMONT                                     1177
SUNSET                                      1156
8TH        

In [72]:
# removing starting and ending spaces 
la_data["location"] = la_data["location"].str.strip()

In [73]:
# replacing multiple spaces with a single one
la_data["location"] = [re.sub(r"\s+"," ",data)for data in la_data["location"]]

In [74]:
# uppercase
la_data["location"] = la_data["location"].str.upper()

In [75]:
la_data["location"].value_counts().head(60)

location
6TH ST                      2571
7TH ST                      2165
9300 TAMPA AV               2069
6TH                         1954
7TH                         1880
5TH ST                      1843
5TH                         1768
6600 TOPANGA CANYON BL      1717
VERMONT AV                  1576
WESTERN AV                  1445
6800 HOLLYWOOD BL           1414
14000 RIVERSIDE DR          1398
500 S SAN PEDRO ST          1398
BROADWAY                    1393
HOLLYWOOD                   1390
3RD ST                      1285
700 W 7TH ST                1278
FIGUEROA                    1183
VERMONT                     1177
SUNSET                      1156
8TH ST                      1140
SHERMAN WY                  1136
700 S FIGUEROA ST           1107
FIGUEROA ST                 1100
100 THE GROVE DR            1092
WILSHIRE BL                 1077
800 N ALAMEDA ST            1047
HOLLYWOOD BL                1043
WESTERN                     1027
4TH ST                       956
6

In [76]:
# this column contain various issues like 6th,6th st and so on 
# olympic , olympic bl and many more...

In [77]:
def wrt_ST_with_ordinal(x):
    if re.fullmatch(r"\d+(ST|TH|RD|ND)",x):
        return f"{x} ST"
    else:
        return x
la_data["location"] = la_data["location"].apply(wrt_ST_with_ordinal)

In [78]:
la_data["location"].value_counts().head(60)

location
6TH ST                      4525
7TH ST                      4045
5TH ST                      3611
3RD ST                      2179
9300 TAMPA AV               2069
8TH ST                      1923
6600 TOPANGA CANYON BL      1717
4TH ST                      1695
VERMONT AV                  1576
WESTERN AV                  1445
6800 HOLLYWOOD BL           1414
14000 RIVERSIDE DR          1398
500 S SAN PEDRO ST          1398
BROADWAY                    1393
HOLLYWOOD                   1390
700 W 7TH ST                1278
1ST ST                      1257
FIGUEROA                    1183
VERMONT                     1177
SUNSET                      1156
SHERMAN WY                  1136
700 S FIGUEROA ST           1107
FIGUEROA ST                 1100
100 THE GROVE DR            1092
WILSHIRE BL                 1077
800 N ALAMEDA ST            1047
HOLLYWOOD BL                1043
WESTERN                     1027
600 S SAN PEDRO ST           935
10200 SANTA MONICA BL        929
P

### Location Column Status:
The data cleaning process revealed several inconsistencies, including mixed formats with full names and abbreviations like "BL" and "BH", inconsistent use of uppercase and lowercase letters, varying string lengths, and the presence of unclear codes such as "BL" and "BH". Additionally, there were potential duplicates with different spellings or formats. While I was able to successfully resolve the ordinal inconsistencies, other issues remain unresolved despite multiple attempts at cleaning the data.



#### changing the datatype of time occ (str to datetime)

In [79]:
la_data['time occ'] = la_data['time occ'].astype('str').str.zfill(4)

In [80]:
la_data['time occ'].sample(15, random_state=42)

1775038    0158
1693812    1200
1753265    1730
1064667    1500
2069291    1634
1836133    0001
2039279    1310
1588066    1830
1147746    1330
1020825    2200
2054848    1830
1123886    0350
1994606    2340
1804447    1800
1867541    1900
Name: time occ, dtype: object

In [81]:
la_data['time occ'].dtype

dtype('O')

#### changing the data type of vict age (string to number) and handle some inconsistencies

In [82]:
la_data['vict age'] = la_data['vict age'].replace({'UnKnown':np.nan})

C:\Users\dell\AppData\Local\Temp\ipykernel_1428\2219287018.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  la_data['vict age'] = la_data['vict age'].replace({'UnKnown':np.nan})


In [83]:
la_data['vict age'].value_counts()

vict age
26.0     24374
25.0     24206
27.0     24184
28.0     24008
29.0     23659
         ...  
95.0       145
96.0       108
97.0       102
98.0        80
118.0        1
Name: count, Length: 99, dtype: int64

In [84]:
la_data['vict age'] = pd.to_numeric(la_data['vict age'])

In [85]:
# replacing the age which is greater than 100 with nan
la_data['vict age'] = la_data['vict age'].mask(la_data['vict age'] > 100 , np.nan) 

#### Premis Desc (Categorization)

In [86]:
premis_list = la_data['premis desc'].unique().tolist()
pd.Series(premis_list).to_csv('premis_list_1.csv',index=False)

In [87]:
def categorize_premis(desc: str) -> str:
    """
    Categorize premises description into a broader category.
    """
    desc = desc.lower() if isinstance(desc, str) else ""

    # ----------------- Residential -----------------
    if any(word in desc for word in [
        "apartment", "house", "residence", "condo", "duplex", "home", 
        "single family dwelling", "project/tenement", "porch", "balcony", 
        "driveway", "yard", "garden", "stairwell", "elevator", "patio",
        "high-rise building"   # newly added
    ]):
        return "Residential"

    # ----------------- Transportation / Vehicle -----------------
    if any(word in desc for word in [
        "station", "bus", "train", "metro", "airport", "terminal", "subway", "parking",
        "vehicle", "taxi", "tow yard", "delivery service", "garage/carport",
        "mta", "red line", "blue line", "expo line", "green line", "truck", "aircraft",
        "slips/dock", "marina", "boat", "tunnel", "redline entrance/exit"
    ]):
        return "Transportation"

    # ----------------- Commercial -----------------
    if any(word in desc for word in [
        "store", "restaurant", "shop", "bar", "club", "cafe", "hotel", "motel",
        "business", "market", "mini-mart", "office building", "laundromat", "cleaner", 
        "hardware", "auto dealership", "nail salon", "computer services", "massage parlor",
        "check cashing", "public storage", "storage shed", "car wash", "mail box",
        "studio", "arcade", "tattoo", "delivery service", "beverly connection", "beverly center",
        "drive thru", "valet", "pay phone", "swap meet", "gun/sporting goods", "tv/radio/appliance",
        "cyberspace", "website", "telecommunication", "short-term vacation rental", "escalator"
    ]):
        return "Commercial"

    # ----------------- Educational -----------------
    if any(word in desc for word in [
        "school", "college", "university", "academy", "kindergarten", "campus", "day care"
    ]):
        return "Educational"

    # ----------------- Public / Government -----------------
    if any(word in desc for word in [
        "library", "courthouse", "police", "fire station", "post office", "city hall", 
        "government", "detention/jail", "missions/shelters", "public restroom", "federal", "state", "county",
        "dept of defense", "nuclear facility"
    ]):
        return "Public/Government"

    # ----------------- Outdoor / Street -----------------
    if any(word in desc for word in [
        "street", "alley", "park", "sidewalk", "freeway", "highway", "field", "lot", "beach",
        "pool", "golf course", "transient encampment", "river bed", "underpass", "pedestrian",
        "dam", "reservoir", "basketball courts", "handball courts", "skating rink",
        "trash can", "trash dumpster"
    ]):
        return "Outdoor/Street"

    # ----------------- Healthcare -----------------
    if any(word in desc for word in [
        "hospital", "clinic", "pharmacy", "medical", "nursing home", "urgent care", 
        "health spa", "vision care", "hospice", "mortuary"
    ]):
        return "Healthcare"

    # ----------------- Financial -----------------
    if any(word in desc for word in [
        "bank", "atm", "credit union", "financial", "exchange", "check cashing", "savings & loan", "finance company"
    ]):
        return "Financial"

    # ----------------- Religious -----------------
    if any(word in desc for word in [
        "church", "mosque", "temple", "synagogue", "chapel", "place of worship", "cemetary"
    ]):
        return "Religious"

    # ----------------- Industrial / Utility -----------------
    if any(word in desc for word in [
        "factory", "warehouse", "plant", "construction", "utility", "power station", 
        "recycling center", "equipment rental", "oil refinery", "garment manufacturer", 
        "water facility", "sewage", "abatement", "manufacturing company", "tool shed"
    ]):
        return "Industrial/Utility"

    # ----------------- Entertainment / Events -----------------
    if any(word in desc for word in [
        "theatre", "stadium", "coliseum", "convention center", "the grove", "staples center", 
        "sports venue", "dodger stadium", "museum", "cultural", "sports arena", "hockey rink",
        "mass gathering location"   # newly added
    ]):
        return "Entertainment"

    # ----------------- Other -----------------
    return "Other"


In [88]:
la_data['premis_category'] = la_data['premis desc'].apply(categorize_premis)

In [89]:
la_data['premis_category'].value_counts()

premis_category
Residential           382699
Outdoor/Street        317689
Transportation        218351
Commercial            110689
Educational            16596
Public/Government       8632
Healthcare              7859
Other                   7636
Financial               3886
Industrial/Utility      2758
Religious               2735
Entertainment           2354
Name: count, dtype: int64

In [90]:
premis_other_list = la_data.loc[la_data['premis_category'] == 'Other','premis desc'].unique().tolist()

In [91]:
pd.Series(premis_other_list).nunique()

4

In [92]:
la_data['premis_category'].unique()

array(['Outdoor/Street', 'Residential', 'Commercial', 'Transportation',
       'Healthcare', 'Financial', 'Public/Government',
       'Industrial/Utility', 'Educational', 'Other', 'Religious',
       'Entertainment'], dtype=object)

In [93]:
la_data[la_data['premis_category'] == 'Other']['premis desc'].value_counts().head(10)

premis desc
OTHER PREMISE                               5965
OTHER/OUTSIDE                               1506
UNKNOWN                                      149
RETIRED (DUPLICATE) DO NOT USE THIS CODE      16
Name: count, dtype: int64

In [94]:
premis_cate = la_data['premis_category'].unique().tolist()
premis_cate

['Outdoor/Street',
 'Residential',
 'Commercial',
 'Transportation',
 'Healthcare',
 'Financial',
 'Public/Government',
 'Industrial/Utility',
 'Educational',
 'Other',
 'Religious',
 'Entertainment']

In [95]:
len_premis_cate = len(premis_cate)

In [96]:
for i in range(len_premis_cate):
    print("PREMIS CATEGORY",premis_cate[i])
    print(la_data[la_data['premis_category'] == premis_cate[i]]['premis desc'].value_counts().head(60))

PREMIS CATEGORY Outdoor/Street
premis desc
STREET                                   244916
SIDEWALK                                  54335
PARK/PLAYGROUND                            7787
ALLEY                                      7393
BEACH                                       876
AUTO SALES LOT                              519
FREEWAY                                     467
TRANSIENT ENCAMPMENT                        283
VACANT LOT                                  262
GOLF COURSE*                                150
POOL-PUBLIC/OUTDOOR OR INDOOR*              138
UNDERPASS/BRIDGE*                           103
BOWLING ALLEY*                              100
SKATEBOARD FACILITY/SKATEBOARD PARK*         87
RIVER BED*                                   74
PEDESTRIAN OVERCROSSING                      60
BASKETBALL COURTS                            55
AMUSEMENT PARK*                              19
DAM/RESERVOIR                                17
SKATING RINK*                                

In [106]:
la_data.isnull().sum()

dr_no                   0
date rptd               0
date occ                0
time occ                0
area name               0
part 1-2                0
crm cd desc             0
vict age           199426
vict sex                0
vict descent            0
premis desc             0
weapon desc             0
status                  0
status desc             0
weapon used             0
weapon_category         0
premis_category         0
datetime occ            0
dtype: int64

In [107]:
la_data.dtypes

dr_no                       int64
date rptd                  object
date occ           datetime64[ns]
time occ                   object
area name                  object
part 1-2                   object
crm cd desc                object
vict age                  float64
vict sex                   object
vict descent               object
premis desc                object
weapon desc                object
status                     object
status desc                object
weapon used                object
weapon_category            object
premis_category            object
datetime occ       datetime64[ns]
dtype: object

In [99]:
la_data.drop("location",axis=1,inplace=True)

In [102]:
la_data['datetime occ'] = pd.to_datetime(la_data['date occ'].dt.strftime("%Y-%m-%d")+' '+la_data['time occ'],format = "%Y-%m-%d %H%M")

In [105]:
la_data.columns

Index(['dr_no', 'date rptd', 'date occ', 'time occ', 'area name', 'part 1-2',
       'crm cd desc', 'vict age', 'vict sex', 'vict descent', 'premis desc',
       'weapon desc', 'status', 'status desc', 'weapon used',
       'weapon_category', 'premis_category', 'datetime occ'],
      dtype='object')

In [103]:
la_data.to_csv('cleaned_LA_file.csv',index=False)